In [1]:
import numpy as np

In [6]:
BERTH = 10
REVENUE_C = 0.1
REVENUE_T = 0.3
PUNISHMENT_C = 0.2
PUNISHMENT_T = 0.4

p_C = radio_ARRIVAL * 0.52
p_T = 0.52 - p_C
p_deC = p_C - 0.02
p_deT = p_T - 0.02

radio_REVENUE = 1.6 # > 0
radio_ARRIVAL = 0.75 # (0, 1)
radio_interval = 0.65

p_C_u = p_C * (1 + radio_interval)
p_T_u = p_T * (1 + radio_interval)
p_deC_u = p_deC * (1 + radio_interval)
p_deT_u = p_deT * (1 + radio_interval)

p_C_l = p_C * (1 - radio_interval)
p_T_l = p_T * (1 - radio_interval)
p_deC_l = p_deC * (1 - radio_interval)
p_deT_l = p_deT * (1 - radio_interval)

In [7]:
action = np.ones((BERTH + 1, BERTH + 1))
r = np.zeros((BERTH + 1, BERTH + 1, 2))
p_u = np.zeros((BERTH + 1, BERTH + 1, 2))
p_l = np.zeros((BERTH + 1, BERTH + 1, 2))

def discharge(y):
  z = REVENUE_C - (REVENUE_C - REVENUE_T) * y / BERTH
  return z

def initial_r():
    for x in range(BERTH):
        for y in range(BERTH):
            for a in range(1):
                if x + y < BERTH and a == 0:
                    r[x,y,a] = p_c * REVENUE_C + p_t * REVENUE_T
                elif x + y == BERTH and y >= 0 and a == 0:
                    r[x,y,a] = p_c * (-PUNISHMENT_C) + p_t * (-PUNISHMENT_T)
                elif x + y == BERTH and y > 0 and a == 1:
                    r[x,y,a] = p_c * REVENUE_C + p_t * REVENUE_T - discharge(y)
                    
def initial_p_u():
    for x in range(BERTH):
        for y in range(BERTH):
            for a in range(1):
                if x + y < BERTH:
                    p_u[x+1,y,a] = p_C_u
                    p_u[x,y+1,a] = p_T_u
                    p_u[x-1,y,a] = MU_u * x
                    p_u[x,y-1,a] = MU_u * y
                    p_u[x,  y,a] = 1 - p_C_u - p_T_u - MU_u * x - MU_u * y
                elif x + y == BERTH and y == 0:
                    p_u[x-1,y,a] = MU_u * x
                    p_u[x  ,y,a] = 1 - MU_u * x
                elif x + y == BERTH and y > 0:
                    p_u[x+1,y-1,a] = p_C_u * a
                    p_u[x-1,  y,a] = MU_u * x
                    p_u[x  ,y-1,a] = MU_u * y
                    p_u[x  ,y  ,a] = 1- p_C_u * a - MU_u * x - MU_u * y
                    
def initial_p_l():
    for x in range(BERTH):
        for y in range(BERTH):
            for a in range(1):
                if x + y < BERTH:
                    p_l[x+1,y,a] = p_C_l
                    p_l[x,y+1,a] = p_T_l
                    p_l[x-1,y,a] = MU_l * x
                    p_l[x,y-1,a] = MU_l * y
                    p_l[x,  y,a] = 1 - p_C_l - p_T_l - MU_l * x - MU_l * y
                elif x + y == BERTH and y == 0:
                    p_l[x-1,y,a] = MU_l * x
                    p_l[x  ,y,a] = 1 - MU_l * x
                elif x + y == BERTH and y > 0:
                    p_l[x+1,y-1,a] = p_C_l * a
                    p_l[x-1,  y,a] = MU_l * x
                    p_l[x  ,y-1,a] = MU_l * y
                    p_l[x  ,y  ,a] = 1- p_C_l * a - MU_l * x - MU_l * y

In [ ]:
def value_iteration(p_c, p_t, theta=0.0001, discount_factor=1.0):
    
    def one_step_likeahead(V, x, y):
        A = np.zeros(2)
        for a in range(1):
            next_value = 0
            tmp_1 = 1. - np.sum(p_l[:,:,a])
            v1 = p_u[:, :, a] - p_l[:, :, a]
            tmp_3 = np.sum(np.multiply(V, p_l[:,:,a]))
            comapre = np.zeros(V.shape)
            for x in range(BERTH):
                for y in range(BERTH):
#                     tmp_1 = 0
#                     tmp_2 = 0
#                     tmp_3 = 0
#                     for i in range(BERTH):
#                         for j in range(BERTH):
#                             tmp_1 += p_l[i,j,a]
#                             tmp_2 +=  (p_u[i,j,a] - p_l[i,j,a]) * min((V[i,j] - V[x,y]), 0)
#                             tmp_3 += V[i,j] * p_l[i,j,a]
#                     A[a] = max(A[a], (1 - tmp_1) * V[x,y] + tmp_2 + tmp_3)
                    v2 = np.minimum((V - V[x,y]), comapre)
                    tmp_2 = np.sum(np.multiply(v1, v2))
                    if x == 0 and y == 0:
                        next_value = tmp_1 * V[x,y] + tmp_2 + tmp_3
                    next_value = max(next_value, tmp_1 * V[x,y] + tmp_2 + tmp_3)
            A[a] = r[x,y,a] + discount_factor * next_value
        return A
    
    V = np.zeros((BERTH + 1, BERTH + 1))
    
    while True:
        delta = 0
        for x in range(BERTH):
            for y in range(BERTH):
                A = one_step_likeahead(V, x, y)
                best_action = max(A)
                delta = max(delta, np.abs(best_action - V[x,y]))
                V[x,y] = best_action
        if delta < theta:
            break
    
#     policy = np.zeros((BERTH + 1, BERTH + 1))
    for x in range(BERTH):
        for y in range(BERTH):
            if x + y == BERTH and y > 0:
                A = one_step_likeahead(V, x, y)
                if np.argmax(A) == 0:
                    action[x][y] = 3
                else:
                    action[x][y] = 2